In [2]:
# python -m pip install -–upgrade pip
# pip install nba_api
# pip install pandas

     -------------------------------------- 255.2/255.2 kB 7.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [23]:
import pandas as pd
import pprint as pp
import json

In [26]:
# career stats

from nba_api.stats.endpoints import PlayerCareerStats
career = PlayerCareerStats(player_id='203999')
# career.get_json()
# career.get_dict()
df = career.get_data_frames()[0]
df


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203999,2015-16,00,1610612743,DEN,21.0,80,55,1733.0,307,...,0.811,181,379,560,189,79,50,104,208,796
1,203999,2016-17,00,1610612743,DEN,22.0,73,59,2038.0,494,...,0.825,212,506,718,359,61,55,171,214,1221
2,203999,2017-18,00,1610612743,DEN,23.0,75,73,2443.0,504,...,0.850,195,608,803,458,90,61,210,212,1385
3,203999,2018-19,00,1610612743,DEN,24.0,80,80,2504.0,616,...,0.821,228,637,865,580,108,55,248,228,1604
4,203999,2019-20,00,1610612743,DEN,25.0,73,73,2335.0,565,...,0.817,166,545,711,512,85,44,226,222,1456
5,203999,2020-21,00,1610612743,DEN,26.0,72,72,2488.0,732,...,0.868,205,575,780,599,95,48,222,192,1898
6,203999,2021-22,00,1610612743,DEN,27.0,74,74,2476.0,764,...,0.810,206,813,1019,584,109,63,281,191,2004
7,203999,2022-23,00,1610612743,DEN,27.0,3,3,107.0,25,...,0.864,6,26,32,29,4,1,11,10,72


In [61]:
# still working on this

from nba_api.stats.endpoints import playerprofilev2

profile = playerprofilev2(player_id='203999')
df = profile.get_data_frames()[0]
df

TypeError: 'module' object is not callable

In [60]:
## get team names

from nba_api.stats.static import teams
nba_teams = teams.get_teams()
print('Number of teams fetched: {}'.format(len(nba_teams)))
nba_teams[:30]
df = pd.DataFrame(nba_teams)
df

Number of teams fetched: 30


,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [57]:
## get player info
from nba_api.stats.endpoints import commonplayerinfo

player_info = commonplayerinfo.CommonPlayerInfo(player_id='1629012')
player_info_df = player_info.get_data_frames()[0]
player_info_df

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,1629012,Collin,Sexton,Collin Sexton,"Sexton, Collin",C. Sexton,collin-sexton,1999-01-04T00:00:00,Alabama,USA,...,collin_sexton,2018,2022,N,Y,Y,2018,1,8,N


In [35]:
## players and IDs

from nba_api.stats.static import players

players = players.get_players()
players

player_ids = [x['id'] for x in players]
player_ids


[76001,
 76002,
 76003,
 51,
 1505,
 949,
 76005,
 76006,
 76007,
 203518,
 1630173,
 101165,
 76008,
 76009,
 76010,
 203112,
 76011,
 76012,
 200801,
 1629121,
 203919,
 149,
 203500,
 912,
 1628389,
 1629061,
 76015,
 202399,
 201167,
 1630534,
 200772,
 76016,
 201336,
 76017,
 201582,
 76018,
 203006,
 1629152,
 202374,
 76019,
 76020,
 1630583,
 203128,
 202332,
 200746,
 76021,
 1626146,
 724,
 2042,
 76022,
 201570,
 1629734,
 1630234,
 2349,
 1629638,
 76024,
 1628959,
 76028,
 1628960,
 1628386,
 706,
 1628443,
 202730,
 76027,
 2124,
 76025,
 951,
 2754,
 76029,
 200984,
 76030,
 201165,
 308,
 1747,
 1824,
 1630631,
 680,
 732,
 202329,
 200811,
 76034,
 2365,
 2431,
 101187,
 202079,
 76035,
 76036,
 1507,
 76037,
 944,
 246,
 202341,
 76040,
 1626147,
 72,
 76041,
 203937,
 76042,
 76043,
 98,
 76045,
 76046,
 201583,
 1000,
 335,
 76048,
 101149,
 76049,
 1628387,
 76050,
 1512,
 203507,
 1628961,
 203648,
 2546,
 1630175,
 21,
 201202,
 203544,
 1628384,
 203951,
 2737,

In [59]:
## get player info for all player IDs
## still working on this

from nba_api.stats.endpoints import commonplayerinfo

name = []
birthdate = []
school = []
country = []
from_year = []
to_year = []
draft_year = []
draft_number = []


for i in player_ids[:3]:

    record = commonplayerinfo.CommonPlayerInfo(player_id=i)
    df = record.get_json()
    df = json.loads(df)

print(list(player_info_df))


['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG', 'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT', 'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS', 'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE', 'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG']
